In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
from dataloader import load_nosepoint, get_gender

In [69]:
get_gender(30)

'f'

In [70]:
load_nosepoint(person_idx=30, viewpt_idx=1)

array([0.5542087 , 0.62599987], dtype=float32)

In [32]:
# Use all 6 images of the first 32 persons (index 1-32) as the training set
# (total 32 x 6 = 192 images)
for person_index in range(1,32+1):
    for viewpoint_index in range(1,6+1):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-32-25ab9ad5bf52>, line 5)

In [ ]:
# Use images of the remaining 8 persons (index 33-40) as the validation set
# (total 8 * 6 = 48 images)
training_set = []